In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = 'True')

Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd

import re
import string
import csv
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Dense, Input, Embedding, Concatenate, TimeDistributed, Bidirectional, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
! pip install rouge
from rouge import Rouge
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Importing package and summarizer for TextRank
import gensim
from gensim.summarization import summarize

# Importing the sumy package
!pip install sumy
import sumy
# Importing the parser and tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
# Importing the LexRank summarizer
from sumy.summarizers.lex_rank import LexRankSummarizer

# Importing the Lsa summarizer
from sumy.summarizers.lsa import LsaSummarizer

# Importing the Luhn summarizer
from sumy.summarizers.luhn import LuhnSummarizer

# Importing the KL-Sum summarizer
from sumy.summarizers.kl import KLSummarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
result_extractive_nltk_df = pd.read_csv('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/result_extractive_nltk.csv')

In [ ]:
result_extractive_sumy_kl_sum_df = pd.read_csv('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/result_extractive_sumy_kl-sum.csv')

In [ ]:
result_extractive_sumy_lexrank_df = pd.read_csv('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/result_extractive_sumy_lexrank.csv')

In [ ]:
result_extractive_sumy_lsa_df = pd.read_csv('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/result_extractive_sumy_lsa.csv')

In [ ]:
result_extractive_sumy_luhn_df = pd.read_csv('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/result_extractive_sumy_luhn.csv')

In [ ]:
result_extractive_textrank_df = pd.read_csv('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/result_extractive_textrank.csv')

In [ ]:
result_extractive_nltk_df.head()

,Article,Original Summary,Model Output
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,"This week, a U.S consumer advisory group set ..."
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,"Rahul Kumar, 17, clambered over the enclosure..."
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,Freedman has stabilised Forest since he repla...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,Liverpool target Neto is also wanted by PSG a...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...","The former Olympian and reality TV star, 65, ..."


In [ ]:
result_extractive_sumy_kl_sum_df.head()

,Article,Original Summary,Model Output
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,While most airlines stick to a pitch of 31 i...
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,A drunk teenage boy had to be rescued by sec...
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,Dougie Freedman is on the verge of agreeing ...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,The Reds were linked with a move for the 25-...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...",He also split from with his wife of more tha...


In [ ]:
result_extractive_sumy_lexrank_df.head()

,Article,Original Summary,Model Output
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,"More than squabbling over the arm rest, shrink..."
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,"Next level drunk: Intoxicated Rahul Kumar, 17,..."
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,Dougie Freedman is set to sign a new deal at N...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,Liverpool target Neto is also wanted by PSG an...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...","The former Olympian and reality TV star, 65, w..."


In [ ]:
result_extractive_sumy_lsa_df.head()

,Article,Original Summary,Model Output
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,With increasing numbers of people taking to ...
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,Mr Kumar explained afterwards that he was dr...
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,Dougie Freedman is on the verge of agreeing ...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,Neto rushes from his goal as Juan Iturbe bea...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...",Bruce Jenner will break his silence in a two...


In [ ]:
result_extractive_sumy_luhn_df.head()

,Article,Original Summary,Model Output
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,"This week, a U.S consumer advisory group set..."
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,A drunk teenage boy had to be rescued by sec...
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,Dougie Freedman is on the verge of agreeing ...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,Liverpool target Neto is also wanted by PSG ...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...","Speaking out: Bruce Jenner, pictured on 'Kee..."


In [ ]:
result_extractive_textrank_df.head()

,Article,Original Summary,Model Output
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,"More than squabbling over the arm rest, shrink..."
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,"Next level drunk: Intoxicated Rahul Kumar, 17,..."
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,Dougie Freedman is set to sign a new deal at N...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,Liverpool target Neto is also wanted by PSG an...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...","The former Olympian and reality TV star, 65, w..."


In [ ]:
result_extractive_nltk_df.fillna("", inplace =True)
rouge = Rouge()
rouge.get_scores(result_extractive_nltk_df['Model Output'], result_extractive_nltk_df['Original Summary'], avg=True, ignore_empty=True)

{'rouge-1': {'r': 0.56521753051062,
  'p': 0.18613200757035214,
  'f': 0.26560001490028456},
 'rouge-2': {'r': 0.24340476099815211,
  'p': 0.0672188943622949,
  'f': 0.09838800436679603},
 'rouge-l': {'r': 0.5263829792002735,
  'p': 0.17287314515526106,
  'f': 0.24681781492635868}}

In [ ]:
results=[]
models = [result_extractive_nltk_df, result_extractive_sumy_kl_sum_df,result_extractive_sumy_lexrank_df,
          result_extractive_sumy_lsa_df, result_extractive_sumy_luhn_df, result_extractive_textrank_df]
rouge = Rouge()
for i in models:
  i.fillna("", inplace=True)
  results.append(rouge.get_scores(i['Model Output'], i['Original Summary'], avg=True, ignore_empty=True))

In [ ]:
models_name = ['result_extractive_nltk_df', 'result_extractive_sumy_kl_sum_df','result_extractive_sumy_lexrank_df',
          'result_extractive_sumy_lsa_df', 'result_extractive_sumy_luhn_df ', 'result_extractive_textrank_df']

results_dict={}
for i in range(6):
  results_dict[models_name[i]]=results[i]

for key,value in results_dict.items():
  print(key, value)

result_extractive_nltk_df {'rouge-1': {'r': 0.56521753051062, 'p': 0.18613200757035214, 'f': 0.26560001490028456}, 'rouge-2': {'r': 0.24340476099815211, 'p': 0.0672188943622949, 'f': 0.09838800436679603}, 'rouge-l': {'r': 0.5263829792002735, 'p': 0.17287314515526106, 'f': 0.24681781492635868}}
result_extractive_sumy_kl_sum_df {'rouge-1': {'r': 0.3045547012403291, 'p': 0.25277902999958357, 'f': 0.2679841189643716}, 'rouge-2': {'r': 0.1049097833856343, 'p': 0.08220217029919319, 'f': 0.08854593776224179}, 'rouge-l': {'r': 0.2783334018764491, 'p': 0.23117949687886763, 'f': 0.24497564767470087}}
result_extractive_sumy_lexrank_df {'rouge-1': {'r': 0.5171325038487575, 'p': 0.25664303486141626, 'f': 0.32014336361809004}, 'rouge-2': {'r': 0.22342437540480403, 'p': 0.09953016954016024, 'f': 0.12526628698951794}, 'rouge-l': {'r': 0.47837707259828116, 'p': 0.23667107031398604, 'f': 0.295716340945698}}
result_extractive_sumy_lsa_df {'rouge-1': {'r': 0.32210567517813055, 'p': 0.23957170793687943, 'f